In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import shapiro

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [67]:
df = pd.read_csv('cont_bcn_eixample_18_23_ML.csv')

df

,year,month,day,CO,NO,NO2,O3,PM10,SO2,season_spring,season_summer,season_winter
0,2018,1,1,0.21,10.71,21.25,55.21,16.62,1.00,0,0,1
1,2018,1,2,0.36,20.00,37.42,36.79,16.17,2.00,0,0,1
2,2018,1,3,0.34,17.58,24.04,36.62,15.67,2.04,0,0,1
3,2018,1,4,0.31,15.92,26.96,32.33,17.12,1.96,0,0,1
4,2018,1,5,0.34,21.27,43.50,25.50,12.45,1.68,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2024,2023,12,16,0.37,17.54,45.58,24.33,23.25,3.38,0,0,0
2025,2023,12,17,0.44,21.88,52.92,16.25,25.92,4.38,0,0,0
2026,2023,12,18,1.00,94.96,83.00,4.12,55.17,8.04,0,0,0
2027,2023,12,19,1.04,99.12,69.42,12.38,53.08,8.29,0,0,0


In [72]:
df_prec = pd.read_csv('precipitacionsbarcelonadesde1786.csv')

df_prec

,Any,Precip_Acum_Gener,Precip_Acum_Febrer,Precip_Acum_Marc,Precip_Acum_Abril,Precip_Acum_Maig,Precip_Acum_Juny,Precip_Acum_Juliol,Precip_Acum_Agost,Precip_Acum_Setembre,Precip_Acum_Octubre,Precip_Acum_Novembre,Precip_Acum_Desembre
0,1786,32.8,28.4,84.4,42.3,8.5,38.0,6.8,52.1,62.0,57.0,195.8,114.1
1,1787,136.4,27.4,44.6,76.9,0.0,29.8,34.7,14.9,205.8,12.4,42.2,0.0
2,1788,9.9,14.9,32.2,12.4,7.5,146.3,59.4,29.8,123.9,163.6,106.6,22.3
3,1789,12.4,12.4,23.6,9.9,32.2,9.9,6.3,69.4,18.7,52.1,18.7,76.9
4,1790,44.6,1.2,188.5,74.4,44.7,39.7,65.7,32.3,71.9,205.8,72.0,14.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,2018,63.6,104.6,112.5,83.0,30.3,53.1,51.5,36.5,44.4,201.9,201.8,4.8
233,2019,18.2,4.6,0.3,36.2,109.0,19.3,75.8,23.1,71.5,83.2,39.4,119.2
234,2020,89.3,2.8,61.6,258.7,29.2,69.1,23.1,12.4,60.2,23.1,52.5,41.5
235,2021,26.2,26.5,7.7,62.7,13.4,7.8,9.0,8.3,23.3,63.2,75.9,3.8


In [73]:
df_prec = df_prec.loc[df_prec['Any'] > 2017]

In [74]:
df_prec = df_prec.reset_index(drop=True)

df_prec

,Any,Precip_Acum_Gener,Precip_Acum_Febrer,Precip_Acum_Marc,Precip_Acum_Abril,Precip_Acum_Maig,Precip_Acum_Juny,Precip_Acum_Juliol,Precip_Acum_Agost,Precip_Acum_Setembre,Precip_Acum_Octubre,Precip_Acum_Novembre,Precip_Acum_Desembre
0,2018,63.6,104.6,112.5,83.0,30.3,53.1,51.5,36.5,44.4,201.9,201.8,4.8
1,2019,18.2,4.6,0.3,36.2,109.0,19.3,75.8,23.1,71.5,83.2,39.4,119.2
2,2020,89.3,2.8,61.6,258.7,29.2,69.1,23.1,12.4,60.2,23.1,52.5,41.5
3,2021,26.2,26.5,7.7,62.7,13.4,7.8,9.0,8.3,23.3,63.2,75.9,3.8
4,2022,12.3,1.9,89.5,38.9,20.0,9.5,2.7,67.9,13.1,11.3,7.6,33.0


In [75]:
df_prec = pd.melt(df_prec, id_vars=['Any'], var_name='Tipo_Precipitacion', value_name='Precipitacion_Acumulada')
df_prec

,Any,Tipo_Precipitacion,Precipitacion_Acumulada
0,2018,Precip_Acum_Gener,63.6
1,2019,Precip_Acum_Gener,18.2
2,2020,Precip_Acum_Gener,89.3
3,2021,Precip_Acum_Gener,26.2
4,2022,Precip_Acum_Gener,12.3
5,2018,Precip_Acum_Febrer,104.6
6,2019,Precip_Acum_Febrer,4.6
7,2020,Precip_Acum_Febrer,2.8
8,2021,Precip_Acum_Febrer,26.5
9,2022,Precip_Acum_Febrer,1.9


In [76]:
df_prec = df_prec.sort_values(by='Any')
df_prec = df_prec.reset_index(drop=True)
df_prec.head(10)

,Any,Tipo_Precipitacion,Precipitacion_Acumulada
0,2018,Precip_Acum_Gener,63.6
1,2018,Precip_Acum_Setembre,44.4
2,2018,Precip_Acum_Maig,30.3
3,2018,Precip_Acum_Juliol,51.5
4,2018,Precip_Acum_Abril,83.0
5,2018,Precip_Acum_Desembre,4.8
6,2018,Precip_Acum_Juny,53.1
7,2018,Precip_Acum_Marc,112.5
8,2018,Precip_Acum_Novembre,201.8
9,2018,Precip_Acum_Agost,36.5


In [79]:
tipo_precipitacion_mapping = {
    'Precip_Acum_Gener': 1,
    'Precip_Acum_Febrer': 2,
    'Precip_Acum_Marc': 3,
    'Precip_Acum_Abril': 4,
    'Precip_Acum_Maig': 5,
    'Precip_Acum_Juny': 6,
    'Precip_Acum_Juliol': 7,
    'Precip_Acum_Agost': 8,
    'Precip_Acum_Setembre': 9,
    'Precip_Acum_Octubre': 10,
    'Precip_Acum_Novembre': 11,
    'Precip_Acum_Desembre': 12
}

df_prec['Mes'] = df_prec['Tipo_Precipitacion'].map(tipo_precipitacion_mapping)
df_prec.head(10)

,Any,Tipo_Precipitacion,Precipitacion_Acumulada,Mes
0,2018,Precip_Acum_Gener,63.6,1
10,2018,Precip_Acum_Febrer,104.6,2
7,2018,Precip_Acum_Marc,112.5,3
4,2018,Precip_Acum_Abril,83.0,4
2,2018,Precip_Acum_Maig,30.3,5
6,2018,Precip_Acum_Juny,53.1,6
3,2018,Precip_Acum_Juliol,51.5,7
9,2018,Precip_Acum_Agost,36.5,8
1,2018,Precip_Acum_Setembre,44.4,9
11,2018,Precip_Acum_Octubre,201.9,10


In [80]:
df_prec = df_prec.sort_values(by=['Any', 'Mes'])

df_prec

,Any,Tipo_Precipitacion,Precipitacion_Acumulada,Mes
0,2018,Precip_Acum_Gener,63.6,1
10,2018,Precip_Acum_Febrer,104.6,2
7,2018,Precip_Acum_Marc,112.5,3
4,2018,Precip_Acum_Abril,83.0,4
2,2018,Precip_Acum_Maig,30.3,5
6,2018,Precip_Acum_Juny,53.1,6
3,2018,Precip_Acum_Juliol,51.5,7
9,2018,Precip_Acum_Agost,36.5,8
1,2018,Precip_Acum_Setembre,44.4,9
11,2018,Precip_Acum_Octubre,201.9,10


In [81]:
df_prec = df_prec.drop(columns='Tipo_Precipitacion')

df_prec.head(10)

,Any,Precipitacion_Acumulada,Mes
0,2018,63.6,1
10,2018,104.6,2
7,2018,112.5,3
4,2018,83.0,4
2,2018,30.3,5
6,2018,53.1,6
3,2018,51.5,7
9,2018,36.5,8
1,2018,44.4,9
11,2018,201.9,10


In [82]:
df_prec = df_prec.reset_index(drop=True)
df_prec.head(10)

,Any,Precipitacion_Acumulada,Mes
0,2018,63.6,1
1,2018,104.6,2
2,2018,112.5,3
3,2018,83.0,4
4,2018,30.3,5
5,2018,53.1,6
6,2018,51.5,7
7,2018,36.5,8
8,2018,44.4,9
9,2018,201.9,10


In [84]:
df_prec['Precip_acum'] = df_prec['Precipitacion_Acumulada']
df_prec = df_prec[['Any', 'Mes', 'Precip_acum']]
df_prec.head(5)

,Any,Mes,Precip_acum
0,2018,1,63.6
1,2018,2,104.6
2,2018,3,112.5
3,2018,4,83.0
4,2018,5,30.3


In [85]:
df_temp = pd.read_csv('temperaturesbarcelonadesde1780.csv')

df_temp

,Any,Temp_Mitjana_Gener,Temp_Mitjana_Febrer,Temp_Mitjana_Marc,Temp_Mitjana_Abril,Temp_Mitjana_Maig,Temp_Mitjana_Juny,Temp_Mitjana_Juliol,Temp_Mitjana_Agost,Temp_Mitjana_Setembre,Temp_Mitjana_Octubre,Temp_Mitjana_Novembre,Temp_Mitjana_Desembre
0,1780,6.7,7.3,11.6,11.4,16.3,19.1,21.4,22.0,20.3,16.3,9.2,7.3
1,1781,7.5,8.9,10.9,13.7,16.8,18.0,22.0,22.6,19.1,15.0,11.8,10.3
2,1782,8.2,6.1,9.8,10.5,15.4,20.1,22.3,22.6,19.9,13.8,7.4,7.5
3,1783,8.3,8.6,9.6,12.4,15.0,19.1,22.9,21.4,18.6,15.5,11.0,10.4
4,1784,7.3,7.1,10.8,10.7,16.7,20.7,22.0,20.5,20.0,13.8,8.5,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,2018,10.5,6.7,10.8,14.7,17.1,21.5,25.3,25.8,22.5,17.0,12.4,11.1
239,2019,8.1,11.9,13.5,13.4,15.6,21.9,25.4,25.1,21.8,18.5,11.9,11.2
240,2020,10.0,12.8,11.9,14.3,19.4,20.1,25.0,25.5,21.7,16.4,14.7,9.3
241,2021,7.7,11.6,12.1,12.9,17.3,23.3,24.8,24.5,23.0,18.1,11.3,10.9


In [86]:
df_temp = df_temp.loc[df_temp['Any'] > 2017]

df_temp = df_temp.reset_index(drop=True)

df_temp = pd.melt(df_temp, id_vars=['Any'], var_name='Tipo_Temp', value_name='Temp_Media')

df_temp = df_temp.sort_values(by='Any')
df_temp = df_temp.reset_index(drop=True)

df_temp

,Any,Tipo_Temp,Temp_Media
0,2018,Temp_Mitjana_Gener,10.5
1,2018,Temp_Mitjana_Setembre,22.5
2,2018,Temp_Mitjana_Maig,17.1
3,2018,Temp_Mitjana_Juliol,25.3
4,2018,Temp_Mitjana_Abril,14.7
5,2018,Temp_Mitjana_Desembre,11.1
6,2018,Temp_Mitjana_Juny,21.5
7,2018,Temp_Mitjana_Marc,10.8
8,2018,Temp_Mitjana_Novembre,12.4
9,2018,Temp_Mitjana_Agost,25.8


In [87]:
tipo_temp_mapping = {
    'Temp_Mitjana_Gener': 1,
    'Temp_Mitjana_Febrer': 2,
    'Temp_Mitjana_Marc': 3,
    'Temp_Mitjana_Abril': 4,
    'Temp_Mitjana_Maig': 5,
    'Temp_Mitjana_Juny': 6,
    'Temp_Mitjana_Juliol': 7,
    'Temp_Mitjana_Agost': 8,
    'Temp_Mitjana_Setembre': 9,
    'Temp_Mitjana_Octubre': 10,
    'Temp_Mitjana_Novembre': 11,
    'Temp_Mitjana_Desembre': 12
}

df_temp['Mes'] = df_temp['Tipo_Temp'].map(tipo_temp_mapping)

df_temp.head(10)

,Any,Tipo_Temp,Temp_Media,Mes
0,2018,Temp_Mitjana_Gener,10.5,1
1,2018,Temp_Mitjana_Setembre,22.5,9
2,2018,Temp_Mitjana_Maig,17.1,5
3,2018,Temp_Mitjana_Juliol,25.3,7
4,2018,Temp_Mitjana_Abril,14.7,4
5,2018,Temp_Mitjana_Desembre,11.1,12
6,2018,Temp_Mitjana_Juny,21.5,6
7,2018,Temp_Mitjana_Marc,10.8,3
8,2018,Temp_Mitjana_Novembre,12.4,11
9,2018,Temp_Mitjana_Agost,25.8,8


In [89]:
df_temp = df_temp.sort_values(by=['Any', 'Mes'])

df_temp = df_temp.drop(columns='Tipo_Temp')

df_temp = df_temp.reset_index(drop=True)

df_temp = df_temp[['Any', 'Mes', 'Temp_Media']]

df_temp.head(10)

,Any,Mes,Temp_Media
0,2018,1,10.5
1,2018,2,6.7
2,2018,3,10.8
3,2018,4,14.7
4,2018,5,17.1
5,2018,6,21.5
6,2018,7,25.3
7,2018,8,25.8
8,2018,9,22.5
9,2018,10,17.0


In [90]:
df_pt = pd.merge(df_prec, df_temp, on=['Any', 'Mes'])

df_pt

,Any,Mes,Precip_acum,Temp_Media
0,2018,1,63.6,10.5
1,2018,2,104.6,6.7
2,2018,3,112.5,10.8
3,2018,4,83.0,14.7
4,2018,5,30.3,17.1
5,2018,6,53.1,21.5
6,2018,7,51.5,25.3
7,2018,8,36.5,25.8
8,2018,9,44.4,22.5
9,2018,10,201.9,17.0


In [91]:
df_archivo = "bcn_pre_tem_18_23.csv"
df_pt.to_csv(df_archivo, index=False)